In [ ]:
from agent_configs import RainbowConfig
import gymnasium as gym
import torch
import random
import numpy as np
import torch
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
from utils.utils import HuberLoss
from cfr_utils import EvalWrapper, evaluatebots, WrapperEnv, load_agents, EmptyConf
import pyspiel
import copy
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
from cfr_network import CFRNetwork
import sys

sys.path.append("..")
from dqn.rainbow.rainbow_agent import RainbowAgent

fhp = pyspiel.load_game(
    "universal_poker",
    {
        "numPlayers": 2,
        "numSuits": 4,
        "numRanks": 13,
        "numHoleCards": 2,
        "numBoardCards": "0 3",
        "bettingAbstraction": "fcpa",
        "numRounds": 2,
        "blind": "50 100",
    },
)
leduc = pyspiel.load_game(
    "universal_poker",
    {
        "numPlayers": 2,
        "numSuits": 2,
        "numRanks": 3,
        "numHoleCards": 1,
        "numBoardCards": "0 1",
        "bettingAbstraction": "fcpa",
        "numRounds": 2,
        "blind": "50 100",
    },
)
leducconfig = {"state_representation_size": 16}
fhpconfig = {"state_representation_size": 108}
leducgame = WrapperEnv(leduc)
fhpgame = WrapperEnv(fhp)


# path1 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/linear/1032012/CFR_0.001LR_TEST_Full_FHP_38.pt' # 10%
# path2 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/linear/4000020/CFR_0.001LR_TEST_Full_FHP_144.pt' #40%
# path3 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/linear/6015876/CFR_0.001LR_TEST_Full_FHP_219.pt' # 60%
# path4 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/linear/8014283/CFR_0.001LR_TEST_Full_FHP_297.pt' #80%
# path5 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/linear/10014831/CFR_0.001LR_TEST_Full_FHP_371.pt' #100%
# path6 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/notlinear/1032012/CFR_0.001LR_TEST_Full_FHP_38.pt' # 10%
# path7 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/notlinear/4000020/CFR_0.001LR_TEST_Full_FHP_144.pt' #40%
# path8 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/notlinear/6015876/CFR_0.001LR_TEST_Full_FHP_219.pt' # 60%
# path9 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/notlinear/8014283/CFR_0.001LR_TEST_Full_FHP_297.pt' #80%
# path10 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/notlinear/10014831/CFR_0.001LR_TEST_Full_FHP_371.pt' #100%


# path1 = 'checkpoints/CFR_0.001LR_TEST_MC_FHP/policy/linear/1005275/CFR_0.001LR_TEST_MC_FHP_56.pt'
# path2 = 'checkpoints/CFR_0.001LR_TEST_MC_FHP/policy/linear/4014284/CFR_0.001LR_TEST_MC_FHP_224.pt'
# path3 = 'checkpoints/CFR_0.001LR_TEST_MC_FHP/policy/linear/6001547/CFR_0.001LR_TEST_MC_FHP_332.pt'
# path4 = 'checkpoints/CFR_0.001LR_TEST_MC_FHP/policy/linear/8016903/CFR_0.001LR_TEST_MC_FHP_439.pt'
# path5 = 'checkpoints/CFR_0.001LR_TEST_MC_FHP/policy/linear/10015290/CFR_0.001LR_TEST_MC_FHP_546.pt'


path6 = "checkpoints_1/notlinear/1005275/CFR_0.001LR_TEST_MC_FHP_56.pt"
path7 = "checkpoints_1/notlinear/4014284/CFR_0.001LR_TEST_MC_FHP_224.pt"
path8 = "checkpoints_1/notlinear/6001547/CFR_0.001LR_TEST_MC_FHP_332.pt"
path9 = "checkpoints_1/notlinear/8016903/CFR_0.001LR_TEST_MC_FHP_439.pt"
path10 = "checkpoints_1/notlinear/10015290/CFR_0.001LR_TEST_MC_FHP_546.pt"

# path11 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path12 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path13 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path14 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path15 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path16 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path17 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path18 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path19 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'
# path20 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt'

# path11 = 'checkpoints/CFR_0.001LR_TEST_Full_FHP/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt' # 10%
# path12 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt' # 40%
# path13 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt' # 60%
# path14 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt' # 80%
# path15 = 'checkpoints/CFR_0.001LR_TEST_Full_Leduc/policy/linear/10011696/CFR_0.001LR_TEST_Full_Leduc_490.pt' # 100% 37.2

# linear_paths = [path1, path2, path3, path4, path5]
notlinear_paths = [path6, path7, path8, path9, path10]
# note numbers are wrong 5 is first before
games = [fhpgame]
for game in games:
    if game == leducgame:
        chosen_game = "leduc"
        game_string = "Leduc"
        # path = path1
    elif game == fhpgame:
        chosen_game = "fhp"
        game_string = "FHP"
        # path = path2

    hidden_dim = 128
    input_dim = 16 if chosen_game == "leduc" else 108
    output_dim = 4
    num_players = 2
    replay_buffer_size = 4000000
    minibatch_size = 10000
    steps_per_epoch = 3000
    traversals = 3000
    training_steps = 20000
    lr = 0.0001
    optimizer = None

    p_v_networks = {
        "input_shape": input_dim,
        "output_shape": output_dim,
        "hidden_size": hidden_dim,
        "learning_rate": lr,
        "optimizer": optimizer,
    }
    active_player_obj = ActivePlayer(num_players)
    config = CFRConfig(
        config_dict={
            "network": {
                "policy": p_v_networks,
                "value": p_v_networks,
                "num_players": num_players,
            },
            "replay_buffer_size": replay_buffer_size,
            "minibatch_size": minibatch_size,
            "steps_per_epoch": steps_per_epoch,
            "traversals": traversals,
            "training_steps": training_steps,
            "active_player_obj": active_player_obj,
        },
        game_config={
            "num_players": num_players,
            "observation_space": input_dim,
            "action_space": 4,
        },
    )
    # for i in range(len(linear_paths)):
    #     path = linear_paths[i]
    #     number = i
    #     print("Linear Path: ", path)
    #     print("Number: ", number)

    #     agent1, agent2 = load_agents(path, path, p_v_networks, num_players)
    #     modelselect = CFRAgent(env=game, config=config)

    #     evaled_game = EvalWrapper(
    #         game=game,
    #         agent=agent1,
    #         in_size=input_dim,
    #         out_size=output_dim,
    #         select_model=modelselect,
    #     )

    #     config_dict = {
    #         "dense_layer_widths": [128, 256, 256, 128],
    #         "value_hidden_layer_widths": [128, 128],
    #         "advantage_hidden_layer_widths": [128, 128],
    #         "adam_epsilon": 1e-8,
    #         "learning_rate": 0.002,
    #         "training_steps": 20000,
    #         "per_epsilon": 0.001,
    #         "per_alpha": 0,
    #         "per_beta": 0,
    #         "per_beta_final": 0.5,
    #         "minibatch_size": 256,
    #         "replay_buffer_size": 1000000,
    #         "min_replay_buffer_size": 256,
    #         "transfer_interval": 1024,
    #         "loss_function": KLDivergenceLoss(),
    #         "clipnorm": 0.0,
    #         "discount_factor": 0.99,
    #         "replay_interval": 64,
    #         "eg_epsilon": 1,
    #         "eg_epsilon_final": 0.0,
    #         "eg_epsilon_final_step": 5000,
    #         "eg_epsilon_decay_type": "linear",
    #         "num_minibatches": 4,
    #     }
    #     gameconfig = EmptyConf()
    #     rconfig = RainbowConfig(config_dict, gameconfig)
    #     device = "cuda:0" if torch.cuda.is_available() else "cpu"
    #     model_name = (
    #         "Rainbow_" + game_string + "_agent_" + str(number) + "_linear" + "_MC"
    #     )
    #     print("Model Name: ", model_name)
    #     agent = RainbowAgent(evaled_game, rconfig, name=model_name, device=device)
    #     agent.checkpoint_interval = 500
    #     agent.train()
    #     agent.save_checkpoint()
    #     print("Checkpoint saved for agent: ", model_name)

    for i in range(len(notlinear_paths)):
        path = notlinear_paths[i]
        number = i
        print("Not Linear Path: ", path)
        print("Number: ", number)

        agent1, agent2 = load_agents(path, path, p_v_networks, num_players)
        modelselect = CFRAgent(env=game, config=config)

        evaled_game = EvalWrapper(
            game=game,
            agent=agent1,
            in_size=input_dim,
            out_size=output_dim,
            select_model=modelselect,
        )

        config_dict = {
            "dense_layer_widths": [128, 256, 256, 128],
            "value_hidden_layer_widths": [128, 128],
            "advantage_hidden_layer_widths": [128, 128],
            "adam_epsilon": 1e-8,
            "learning_rate": 0.002,
            "training_steps": 20000,
            "per_epsilon": 0.001,
            "per_alpha": 0,
            "per_beta": 0,
            "per_beta_final": 0.5,
            "minibatch_size": 256,
            "replay_buffer_size": 1000000,
            "min_replay_buffer_size": 256,
            "transfer_interval": 1024,
            "loss_function": KLDivergenceLoss(),
            "clipnorm": 0.0,
            "discount_factor": 0.99,
            "replay_interval": 64,
            "eg_epsilon": 1,
            "eg_epsilon_final": 0.0,
            "eg_epsilon_final_step": 5000,
            "eg_epsilon_decay_type": "linear",
            "num_minibatches": 4,
        }
        gameconfig = EmptyConf()
        rconfig = RainbowConfig(config_dict, gameconfig)
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model_name = (
            "Rainbow_" + game_string + "_agent_" + str(number) + "_notlinear" + "_MC"
        )
        print("Model Name: ", model_name)
        agent = RainbowAgent(evaled_game, rconfig, name=model_name, device=device)
        agent.checkpoint_interval = 500
        agent.train()
        agent.save_checkpoint()
        print("Checkpoint saved for agent: ", model_name)